In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model,Model
from tensorflow.keras.layers import Dense,Conv2D,Dropout,MaxPooling2D,GlobalMaxPooling2D,Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os 
import numpy as np
import time
from PIL import Image

In [2]:
dir_pat='/Users/draculalemon/Downloads/go_stop'

In [3]:
img_class=[]
for class_name in os.listdir(dir_pat):
    class_path=os.path.join(dir_pat,class_name)
    if os.path.isdir(class_path):
        no_of_img=len(os.listdir(class_path))
        img_class.append((class_name))
        print(no_of_img)        

11
11


In [4]:
#now we will spit the data into batches and also use some images generator for better predictio
batch_size=7
IMG_height=200
IMG_WEIGHT=200
split=0.2

datagen_train=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                             validation_split=split,
                                                             rotation_range=20,
                                                             width_shift_range=0.1,
                                                             height_shift_range=0.1,
                                                             shear_range=0.4,zoom_range=0.3,
                                                             horizontal_flip=True,
                                                             vertical_flip=True,
                                                             fill_mode='reflect')

datagen_val=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                            validation_split=split)

In [5]:
#passing the image generator to the file or applying it
train_datagen=datagen_train.flow_from_directory(batch_size=10,directory=dir_pat,
                                               shuffle=True,
                                               subset='training',
                                               interpolation='bicubic',
                                               target_size=(IMG_height,IMG_WEIGHT))

validate_datagen=datagen_val.flow_from_directory(batch_size=batch_size,
                                                directory=dir_pat,
                                                shuffle=True,
                                                subset='validation',
                                                interpolation='bicubic',
                                                target_size=(IMG_height,IMG_WEIGHT))

Found 16 images belonging to 2 classes.
Found 3 images belonging to 2 classes.


In [6]:
# reseting the index after split
validate_datagen.reset()
train_datagen.reset()

In [7]:

model=Sequential([
    Conv2D(16,3,padding='same',activation='relu',input_shape=(IMG_height,IMG_WEIGHT,3)),
    MaxPooling2D(),
    
    Conv2D(32,3,padding='same',activation='relu'),
    MaxPooling2D(),
    
    Conv2D(64,3,padding='same',activation='relu'),
    MaxPooling2D(),
    
    Conv2D(128,3,padding='same',activation='relu'),
    MaxPooling2D(),
    
    GlobalMaxPooling2D(),
    Dense(1000,activation='relu'),
    Dense(len(img_class),activation='softmax')
        
])

Metal device set to: Apple M1


2022-07-17 11:46:12.422267: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-17 11:46:12.422511: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
#using early stoping for preventing overfiting
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=6,mode='min',
                                      restore_best_weights=True)

In [9]:
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',
             metrics=['accuracy'])

In [10]:
start = time.time()
epoches=8
train_model=model.fit(train_datagen,steps_per_epoch=(train_datagen.samples)//batch_size,
                  epochs=epoches,validation_data=validate_datagen,
                  validation_steps=(validate_datagen.samples)//batch_size,
                  callbacks=[early_stopping_callback])
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/8


2022-07-17 11:46:12.832152: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-17 11:46:13.216723: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 192ms/step - loss: 0.7422 - accuracy: 0.3750
Epoch 2/8
2/2 [==============================] - 0s 310ms/step - loss: 0.6348 - accuracy: 0.7500
Epoch 3/8
2/2 [==============================] - 0s 112ms/step - loss: 0.5957 - accuracy: 0.8750
Epoch 4/8
2/2 [==============================] - 0s 335ms/step - loss: 0.4717 - accuracy: 0.9375
Epoch 5/8
2/2 [==============================] - 0s 166ms/step - loss: 0.3482 - accuracy: 0.9375
Epoch 6/8
2/2 [==============================] - 0s 223ms/step - loss: 0.2191 - accuracy: 1.0000
Epoch 7/8
2/2 [==============================] - 0s 235ms/step - loss: 0.1742 - accuracy: 0.9375
Epoch 8/8
2/2 [==============================] - 1s 285ms/step - loss: 0.1327 - accuracy: 0.9375
Training time: 4.8251330852508545s


In [15]:
#saving the model
model.save('arduino_sign.h5')